In [20]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn.functional as F
from torch import nn, optim
import os

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Paths
data_dir = './Data/fits_filtered4/augmented/data/'  # Update this path to your dataset directory
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

# Data Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])

# Load Datasets
train_dataset = ImageFolder(root=train_dir, transform=transform)
val_dataset = ImageFolder(root=val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load Pre-Trained ResNet
model = torchvision.models.resnet50(pretrained=True)

# Modify Output Layer for Binary Classification
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 classes: streak, no_streak
model = model.to(device)

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=4):
    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_loss, correct, total = 0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        train_acc = 100. * correct / total
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%')

        # Validation Phase
        model.eval()
        val_loss, correct, total = 0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        
        val_acc = 100. * correct / total
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}, Validation Acc: {val_acc:.2f}%')

    return model

# Train the Model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)

# Save the Model
torch.save(trained_model.state_dict(), 'streak_detector.pth')

# Inference Function
def predict(image_path, model, transform):
    from PIL import Image
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    output = model(image)
    _, prediction = output.max(1)
    return "streak" if prediction.item() == 0 else "no_streak"

def predict_with_probability(image_path, model, transform):
    from PIL import Image
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)  # Transform and add batch dimension
    output = model(image)  # Get logits
    probabilities = F.softmax(output, dim=1)  # Convert logits to probabilities
    prob_streak, prob_no_streak = probabilities[0].tolist()  # Extract probabilities
    prediction = "streak" if prob_streak > prob_no_streak else "no_streak"
    return prediction, prob_streak, prob_no_streak


# Example Prediction
# image_path = './Data/fits_filtered4/augmented/data/val/streak/tic6_2.png'  # Replace with an actual image path
image_path = './Data/fits_filtered4/data/val/no_streak/tic5.png'  # Replace with an actual image path

prediction, prob_streak, prob_no_streak = predict_with_probability(image_path, trained_model, transform)

print(f"Prediction: {prediction}")
print(f"Probability of Streak: {prob_streak:.2f}")
print(f"Probability of No Streak: {prob_no_streak:.2f}")


# print(f"Prediction: {predict(example_image, trained_model, transform)}")


c:\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Train Loss: 0.7667, Train Acc: 62.16%
Validation Loss: 6.3412, Validation Acc: 73.68%
Epoch 2/10, Train Loss: 0.5701, Train Acc: 74.32%
Validation Loss: 282.3446, Validation Acc: 73.68%
Epoch 3/10, Train Loss: 0.4667, Train Acc: 67.57%
Validation Loss: 4.6882, Validation Acc: 73.68%
Epoch 4/10, Train Loss: 0.2842, Train Acc: 85.14%
Validation Loss: 2.2213, Validation Acc: 68.42%
Epoch 5/10, Train Loss: 0.3338, Train Acc: 79.73%
Validation Loss: 4.3473, Validation Acc: 57.89%
Epoch 6/10, Train Loss: 0.2541, Train Acc: 90.54%
Validation Loss: 4.5643, Validation Acc: 63.16%
Epoch 7/10, Train Loss: 0.2703, Train Acc: 87.84%
Validation Loss: 3.8468, Validation Acc: 73.68%
Epoch 8/10, Train Loss: 0.3609, Train Acc: 81.08%
Validation Loss: 2.0858, Validation Acc: 78.95%
Epoch 9/10, Train Loss: 0.3202, Train Acc: 78.38%
Validation Loss: 0.7060, Validation Acc: 78.95%
Epoch 10/10, Train Loss: 0.3447, Train Acc: 86.49%
Validation Loss: 0.2208, Validation Acc: 89.47%
Prediction: strea